<a href="https://colab.research.google.com/github/TheOsminShow/TheOsminShow/blob/main/Phase_Fixer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font size=6>Phase Fixer for INST Roformers</font><br>
Powered by ZFTurbo's [Music-Source-Separation-Training](https://github.com/ZFTurbo/Music-Source-Separation-Training/)

<font size=1>*Based on the colab provided by [jarredou](https://github.com/jarredou) & deton</font>

In [1]:
#@markdown # Install
import requests
import sys
import os
import time
from IPython.display import clear_output

if not os.path.exists('/content/drive'):
  from google.colab import drive
  drive.mount('/content/drive')

if not os.path.exists('/content/Music-Source-Separation-Training'):
  print('Cloning MSST repository...')
  !git clone -b main https://github.com/lucassantillifuck2fa/Music-Source-Separation-Training
  if not os.path.exists('/content/Music-Source-Separation-Training/ckpts'):
    !mkdir '/content/Music-Source-Separation-Training/ckpts'
  if not os.path.exists('/content/Music-Source-Separation-Training/temp'):
    !mkdir '/content/Music-Source-Separation-Training/temp'

%cd /content
clear_output()

req_text = """
mutagen==1.47.0
ml_collections==1.1.0
numpy>=1.26.0
pandas==2.2.2
scipy
tqdm
segmentation_models_pytorch==0.3.3
timm
audiomentations==0.24.0
pedalboard==0.8.2
omegaconf
beartype
rotary_embedding_torch==0.3.5
einops
librosa
demucs #==4.0.0
# transformers==4.35.0
torchmetrics==0.11.4
spafe==0.3.2
protobuf
torch_audiomentations
asteroid==0.7.0
auraloss
torchseg
"""

with open("Music-Source-Separation-Training/requirements.txt", "w") as f:
    f.write(req_text)

print('Installing the dependencies... This will take a few minutes')
!pip install --no-cache-dir -r 'Music-Source-Separation-Training/requirements.txt' &> /dev/null
print('Installation is done!')

Installing the dependencies... This will take a few minutes
Installation is done!


In [2]:
%cd '/content/Music-Source-Separation-Training/'
clear_output()
import os
import mutagen
import torch
import yaml
from urllib.parse import quote
import subprocess
import time
import sys
from IPython.display import clear_output
extensions = (".wav", ".mp3", ".m4a", ".weba", ".flac", ".ogg", ".mp4", ".webv", ".opus", ".m4v", ".avi", ".mpg", ".mkv")

class IndentDumper(yaml.Dumper):
    def increase_indent(self, flow=False, indentless=False):
        return super(IndentDumper, self).increase_indent(flow, False)

def tuple_constructor(loader, node):
    # Load the sequence of values from the YAML node
    values = loader.construct_sequence(node)
    # Return a tuple constructed from the sequence
    return tuple(values)

# Register the constructor with PyYAML
yaml.SafeLoader.add_constructor('tag:yaml.org,2002:python/tuple',
tuple_constructor)

def conf_edit(config_path, chunk_size, overlap):
    with open(config_path, 'r') as f:
        data = yaml.load(f, Loader=yaml.SafeLoader)

    # handle cases where 'use_amp' is missing from config:
    if 'use_amp' not in data.keys():
      data['training']['use_amp'] = True

    data['audio']['chunk_size'] = chunk_size
    data['inference']['num_overlap'] = overlap

    if data['inference']['batch_size'] == 1:
      data['inference']['batch_size'] = 2

    #print("Using custom overlap and chunk_size values:")
    #print(f"overlap = {data['inference']['num_overlap']}")
    #print(f"chunk_size = {data['audio']['chunk_size']}")
    #print(f"batch_size = {data['inference']['batch_size']}")

    with open(config_path, 'w') as f:
        yaml.dump(data, f, default_flow_style=False, sort_keys=False, Dumper=IndentDumper, allow_unicode=True)

def download_file(url):
    # Encode the URL to handle spaces and special characters
    encoded_url = quote(url, safe=':/')

    path = 'ckpts'
    os.makedirs(path, exist_ok=True)
    filename = os.path.basename(encoded_url)
    file_path = os.path.join(path, filename)

    if os.path.exists(file_path):
        return

    try:
        response = torch.hub.download_url_to_file(encoded_url, file_path)
        print(f"File '{filename}' downloaded successfully")
    except Exception as e:
        print(f"Error downloading file '{filename}' from '{url}': {e}")

#@markdown # Separation
#@markdown *Separation config:*
input = "/content/drive/MyDrive/songs/" # @param ["/content/drive/MyDrive/songs/"] {"allow-input":true}
output_folder = "/content/drive/MyDrive/separated/" # @param ["/content/drive/MyDrive/separated/"] {"allow-input":true}
#@markdown
source_model = 'VOCALS-Melband-Roformer BigBeta6X (by unwa)' #@param ['VOCALS-MelBand-Roformer (by Becruily)', 'VOCALS-Mel-Roformer big beta 4 (by unwa)', 'VOCALS-Melband-Roformer BigBeta5e (by unwa)','VOCALS-Melband-Roformer BigBeta6 (by unwa)', 'VOCALS-Melband-Roformer BigBeta6X (by unwa)', 'VOCALS-MelBand-Roformer (by KimberleyJSN)', 'VOCALS-MelBand-Roformer Kim FT (by Unwa)', 'VOCALS-MelBand-Roformer Kim FT 2 (by Unwa)', 'VOCALS-MelBand-Roformer Kim FT 2 Bleedless (by Unwa)', 'VOCALS-Mel-Roformer FT3 Preview (by unwa)', 'VOCALS-BS-Roformer_1296 (by viperx)', 'VOCALS-BS-Roformer_1297 (by viperx)', 'VOCALS-BS-RoformerLargev1 (by unwa)', 'VOCALS-BS-Roformer Revive (by unwa)']
target_model = 'INST-Mel-Roformer INSTV7 (by Gabox)' #@param ['INST-MelBand-Roformer (by Becruily)', 'INST-Mel-Roformer v1 (by unwa)','INST-Mel-Roformer v2 (by unwa)', 'INST-Mel-Roformer v1e (by unwa)', 'INST-Mel-Roformer v1e+ (by unwa)', 'INST-Mel-Roformer INSTV7 (by Gabox)', 'INST-VOC-Mel-Roformer a.k.a. duality (by unwa)', 'INST-VOC-Mel-Roformer a.k.a. duality v2 (by unwa)', 'INST-MelBand-Roformer inst_gabox3 (by Gabox)']
export_format = 'flac PCM_16' #@param ['wav FLOAT', 'flac PCM_16', 'flac PCM_24']
#@markdown ---
#@markdown *Roformers custom config:*
chunk_size = 352800 # @param ["132300","352800","485100","661500"] {"type":"raw"}
overlap = 15 #@param {type:"slider", min:2, max:40, step:1}
#phasefix_testmode = True # @param {"type":"boolean"}
#@markdown ---
#@markdown *Phase Fixer custom config:*
scale_factor = 1.4 # @param {"type":"slider","min":0.5,"max":3,"step":0.05}
low_cutoff = 500 # @param {"type":"slider","min":100,"max":2000,"step":100}
high_cutoff = 8900 # @param {"type":"slider","min":2000,"max":10000,"step":100}
#@markdown ---
source_inference = True # @param {"type":"boolean"}
target_inference = True # @param {"type":"boolean"}
phase_fixer = True # @param {"type":"boolean"}
copy_metadata = True # @param {"type":"boolean"}
prevent_overwrite = False # @param {"type":"boolean"}


if not os.path.exists(input):
  print(f"Invalid Input! Make sure to input a valid Google Drive Path that points to a folder or audio file.")
  sys.exit()

if output_folder.startswith(r"/content/drive/MyDrive/"):
  if not os.path.exists(output_folder):
    print(f"Trying to create output directory \"{output_folder}\"")
    os.mkdir(output_folder)
    time.sleep(3)
    if os.path.exists(output_folder):
      print(f"Output directory created successfully")
    else:
      print(f"Couldn't create output directory. Make sure to input a valid directory that exists in your Drive.")
      sys.exit()
else:
  print(f"Invalid Output Folder! Make sure to input a valid Google Drive Path, like \"/content/drive/MyDrive/separated)\"")
  sys.exit()

if export_format.startswith('flac'):
    flac_file = True
    pcm_type = export_format.split(' ')[1]
else:
    flac_file = False
    pcm_type = None

if os.path.isfile(input) and input.lower().endswith(extensions):
    # Input is a single file
    all_files = [os.path.basename(input)]
    input_path = os.path.dirname(input)
    print(f"Processing file: {input}")
elif os.path.isdir(input):
    # Input is a directory
    input_path = input
    all_files = [f for f in os.listdir(input_path) if f.lower().endswith(extensions)]
    if not all_files:
        print("No valid audio files found in the directory.")
        sys.exit()
    print(f"Processing {len(all_files)} files from directory: {input}")
else:
    print("Invalid input! Please provide either a valid audio file or a directory containing audio files.")
    sys.exit()

for idx, file in enumerate(all_files, start=1):
    filename = os.path.splitext(file)[0]
    input_file = os.path.join(input_path, file)
    filename_wo_ext = os.path.splitext(os.path.basename(file))[0]
    output_path = f"{output_folder}/{filename_wo_ext} (Fixed Instrumental).flac"
    if source_inference and target_inference and phase_fixer:
      output_base = f"temp/{source_model}/{filename_wo_ext}"
      output_unwa = f"temp/{target_model}/{filename_wo_ext}"
    else:
      output_base = f"{output_folder}/{source_model}/{filename_wo_ext}"
      output_unwa = f"{output_folder}/{target_model}/{filename_wo_ext}"
    if source_inference:
      os.makedirs(output_base, exist_ok=True)
    if target_inference:
      os.makedirs(output_unwa, exist_ok=True)

    if prevent_overwrite:
      if os.path.exists(output_path):
        clear_output(wait=True)
        print(f"{file} already exists in the output directory!")
        time.sleep(4)
        continue

    if source_inference:
      if source_model == 'INST-Mel-Roformer INSTFV8b (by Gabox)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/inst_gabox.yaml'
        start_check_point = 'ckpts/Inst_FV8b.ckpt'
        download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
        download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/experimental/Inst_FV8b.ckpt')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-BS-Roformer_1297 (by viperx)':
        model_type = 'bs_roformer'
        config_path = 'ckpts/model_bs_roformer_ep_317_sdr_12.9755.yaml'
        start_check_point = 'ckpts/model_bs_roformer_ep_317_sdr_12.9755.ckpt'
        download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/viperx/model_bs_roformer_ep_317_sdr_12.9755.yaml')
        download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_317_sdr_12.9755.ckpt')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-BS-Roformer_1296 (by viperx)':
        model_type = 'bs_roformer'
        config_path = 'ckpts/model_bs_roformer_ep_368_sdr_12.9628.yaml'
        start_check_point = 'ckpts/model_bs_roformer_ep_368_sdr_12.9628.ckpt'
        download_file('https://github.com/TRvlvr/model_repo/releases/download/all_public_uvr_models/model_bs_roformer_ep_368_sdr_12.9628.ckpt')
        download_file('https://raw.githubusercontent.com/TRvlvr/application_data/main/mdx_model_data/mdx_c_configs/model_bs_roformer_ep_368_sdr_12.9628.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-MelBand-Roformer (by KimberleyJSN)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_vocals_mel_band_roformer_kj.yaml'
        start_check_point = 'ckpts/MelBandRoformer.ckpt'
        download_file('https://raw.githubusercontent.com/ZFTurbo/Music-Source-Separation-Training/main/configs/KimberleyJensen/config_vocals_mel_band_roformer_kj.yaml')
        download_file('https://huggingface.co/KimberleyJSN/melbandroformer/resolve/main/MelBandRoformer.ckpt')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-MelBand-Roformer Kim FT (by Unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_kimmel_unwa_ft.yaml'
        start_check_point = 'ckpts/kimmel_unwa_ft.ckpt'
        download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/config_kimmel_unwa_ft.yaml')
        download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/kimmel_unwa_ft.ckpt')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-MelBand-Roformer Kim FT 2 Bleedless (by Unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_kimmel_unwa_ft.yaml'
        start_check_point = 'ckpts/kimmel_unwa_ft2_bleedless.ckpt'
        download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/config_kimmel_unwa_ft.yaml')
        download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/kimmel_unwa_ft2_bleedless.ckpt')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-MelBand-Roformer Kim FT 2 (by Unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_kimmel_unwa_ft.yaml'
        start_check_point = 'ckpts/kimmel_unwa_ft2.ckpt'
        download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/config_kimmel_unwa_ft.yaml')
        download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/kimmel_unwa_ft2.ckpt')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-BS-Roformer Revive (by unwa)':
        model_type = 'bs_roformer'
        config_path = 'ckpts/config.yaml'
        start_check_point = 'ckpts/bs_roformer_revive.ckpt'
        download_file('https://huggingface.co/pcunwa/BS-Roformer-Revive/resolve/main/bs_roformer_revive.ckpt')
        download_file('https://huggingface.co/pcunwa/BS-Roformer-Revive/resolve/main/config.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-BS-RoformerLargev1 (by unwa)':
        model_type = 'bs_roformer'
        config_path = 'ckpts/config_bsrofoL.yaml'
        start_check_point = 'ckpts/BS-Roformer_LargeV1.ckpt'
        download_file('https://huggingface.co/jarredou/unwa_bs_roformer/resolve/main/BS-Roformer_LargeV1.ckpt')
        download_file('https://huggingface.co/jarredou/unwa_bs_roformer/raw/main/config_bsrofoL.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-Melband-Roformer BigBeta6X (by unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/big_beta6x.yaml'
        start_check_point = 'ckpts/big_beta6x.ckpt'
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta6x.ckpt')
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta6x.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-Melband-Roformer BigBeta6 (by unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/big_beta6.yaml'
        start_check_point = 'ckpts/big_beta6.ckpt'
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta6.ckpt')
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta6.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-Melband-Roformer BigBeta5e (by unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/big_beta5e.yaml'
        start_check_point = 'ckpts/big_beta5e.ckpt'
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta5e.ckpt')
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/big_beta5e.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-Mel-Roformer big beta 4 (by unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_melbandroformer_big_beta4.yaml'
        start_check_point = 'ckpts/melband_roformer_big_beta4.ckpt'
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/resolve/main/melband_roformer_big_beta4.ckpt')
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-big/raw/main/config_melbandroformer_big_beta4.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-MelBand-Roformer (by Becruily)':
          model_type = 'mel_band_roformer'
          config_path = 'ckpts/config_vocals_becruily.yaml'
          start_check_point = 'ckpts/mel_band_roformer_vocals_becruily.ckpt'
          download_file('https://huggingface.co/becruily/mel-band-roformer-vocals/resolve/main/config_vocals_becruily.yaml')
          download_file('https://huggingface.co/becruily/mel-band-roformer-vocals/resolve/main/mel_band_roformer_vocals_becruily.ckpt')
          conf_edit(config_path, chunk_size, overlap)

      elif source_model == 'VOCALS-Mel-Roformer FT3 Preview (by unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_kimmel_unwa_ft.yaml'
        start_check_point = 'ckpts/kimmel_unwa_ft3_prev.ckpt'
        download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/kimmel_unwa_ft3_prev.ckpt')
        download_file('https://huggingface.co/pcunwa/Kim-Mel-Band-Roformer-FT/resolve/main/config_kimmel_unwa_ft.yaml')
        conf_edit(config_path, chunk_size, overlap)

      #start_source_inference
      clear_output(wait=True)
      print(f"\n[{idx}/{len(all_files)}] Processing \"{file}\" using Source Model \"{source_model}\"")
      !python inference.py \
        --model_type {model_type} \
        --config_path '{config_path}' \
        --start_check_point '{start_check_point}' \
        --input_file "{input_file}" \
        --store_dir "{output_base}" \
        --extract_instrumental \
        {('--flac_file' if flac_file else '')} \
        {('--pcm_type ' + pcm_type if pcm_type else '')}

    if target_inference:
      if target_model == 'INST-Mel-Roformer v1 (by unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_melbandroformer_inst.yaml'
        start_check_point = 'ckpts/melband_roformer_inst_v1.ckpt'
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/melband_roformer_inst_v1.ckpt')
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/raw/main/config_melbandroformer_inst.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif target_model == 'INST-Mel-Roformer v2 (by unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_melbandroformer_inst_v2.yaml'
        start_check_point = 'ckpts/melband_roformer_inst_v2.ckpt'
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/melband_roformer_inst_v2.ckpt')
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/raw/main/config_melbandroformer_inst_v2.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif target_model == 'INST-Mel-Roformer v1e (by unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_melbandroformer_inst.yaml'
        start_check_point = 'ckpts/inst_v1e.ckpt'
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/inst_v1e.ckpt')
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/raw/main/config_melbandroformer_inst.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif target_model == 'INST-VOC-Mel-Roformer a.k.a. duality (by unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_melbandroformer_instvoc_duality.yaml'
        start_check_point = 'ckpts/melband_roformer_instvoc_duality_v1.ckpt'
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/resolve/main/melband_roformer_instvoc_duality_v1.ckpt')
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/raw/main/config_melbandroformer_instvoc_duality.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif target_model == 'INST-VOC-Mel-Roformer a.k.a. duality v2 (by unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_melbandroformer_instvoc_duality.yaml'
        start_check_point = 'ckpts/melband_roformer_instvox_duality_v2.ckpt'
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/resolve/main/melband_roformer_instvox_duality_v2.ckpt')
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-InstVoc-Duality/raw/main/config_melbandroformer_instvoc_duality.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif target_model == 'INST-MelBand-Roformer inst_gabox3 (by Gabox)':
          model_type = 'mel_band_roformer'
          config_path = 'ckpts/inst_gabox.yaml'
          start_check_point = 'ckpts/inst_gabox3.ckpt'
          download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
          download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox3.ckpt')
          conf_edit(config_path, chunk_size, overlap)

      elif target_model == 'INST-MelBand-Roformer (by Becruily)':
          model_type = 'mel_band_roformer'
          config_path = 'ckpts/config_instrumental_becruily.yaml'
          start_check_point = 'ckpts/mel_band_roformer_instrumental_becruily.ckpt'
          download_file('https://huggingface.co/becruily/mel-band-roformer-instrumental/resolve/main/config_instrumental_becruily.yaml')
          download_file('https://huggingface.co/becruily/mel-band-roformer-instrumental/resolve/main/mel_band_roformer_instrumental_becruily.ckpt')
          conf_edit(config_path, chunk_size, overlap)

      elif target_model == 'INST-Mel-Roformer INSTV7 (by Gabox)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/inst_gabox.yaml'
        start_check_point = 'ckpts/Inst_GaboxV7.ckpt'
        download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/Inst_GaboxV7.ckpt')
        download_file('https://huggingface.co/GaboxR67/MelBandRoformers/resolve/main/melbandroformers/instrumental/inst_gabox.yaml')
        conf_edit(config_path, chunk_size, overlap)

      elif target_model == 'INST-Mel-Roformer v1e+ (by unwa)':
        model_type = 'mel_band_roformer'
        config_path = 'ckpts/config_melbandroformer_inst.yaml'
        start_check_point = 'ckpts/inst_v1e_plus.ckpt'
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/inst_v1e_plus.ckpt')
        download_file('https://huggingface.co/pcunwa/Mel-Band-Roformer-Inst/resolve/main/config_melbandroformer_inst.yaml')
        conf_edit(config_path, chunk_size, overlap)

      #start_target_inference
      clear_output(wait=True)
      print(f"\n[{idx}/{len(all_files)}] Processing \"{file}\" using Target Model \"{target_model}\"")
      !python inference.py \
        --model_type {model_type} \
        --config_path '{config_path}' \
        --start_check_point '{start_check_point}' \
        --input_file "{input_file}" \
        --store_dir "{output_unwa}" \
        {('--flac_file' if flac_file else '')} \
        {('--pcm_type ' + pcm_type if pcm_type else '')}

    if phase_fixer:
      if source_inference and target_inference:
          clear_output(wait=True)
          print(f"\n[{idx}/{len(all_files)}] Processing \"{file}\" using Phase Fixer")
          !python torch_colab.py --base_folder "{output_base}" --unwa_folder "{output_unwa}" --low_cutoff {low_cutoff} --high_cutoff {high_cutoff} --scale_factor {scale_factor} --output_folder "{output_folder}"
          time.sleep(1)

    if phase_fixer and copy_metadata:
      if source_inference and target_inference:
        time.sleep(3)
        if os.path.isfile(input_file) and input_file.lower().endswith((".mp3", ".m4a", ".flac")):
          !python copy_metadata.py --source_file "{input_file}" --target_file "{output_path}"
          time.sleep(3)

Invalid Input! Make sure to input a valid Google Drive Path that points to a folder or audio file.


SystemExit: 

/usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


**input:** Can point to a directory or audio file.<br>

**source_inference:** Enables inference for the source model, which extracts the phase data used to correct the target file.<br>
**target_inference:** Enables inference for the target model, which receives the phase correction to remove noise.<br>

**chunk_size:** total of audio samples that will be processed by the model at once. 44100 equals to 1 second.<br>
**overlap:** Number of times that each song will be processed per model, starting from a different sample, to reduce clipping and bleeding. More overlaps means longer inference time per song. Colab might force stop the inference if it runs for too long, setting higher overlap values means less time available to process more songs. 2 to 4 is usually enough.<br>

**scale_factor:** Controls how strongly the high frequencies from the source file influence the corrected file. A higher scale factor results in more blending of source phase into the higher frequencies, making them sound more like the source file.
A lower scale factor keeps more of the target file’s original high-frequency characteristics. Higher values blend higher frequencies with more strength, reducing more noise but can lead to muddiness in the final file.<br>

**low_cutoff:** Increasing this value reduces the influence of the source file on the low frequencies, making the corrected file’s low end sound closer to the target file.<br>
**high_cutoff:** Reducing this value keeps more of the target file’s original phase in the high frequencies, while increasing it allows greater blending of the source file’s phase into those frequencies.<br>

**copy_metadata**: If input format is either mp3, m4a or flac, and export format is flac, metadata from the input will be copied to the fixed instrumental.<br>
**prevent_overwrite**: If a fixed instrumental already exists for the current song it will be skipped from inference loop, to save time.
